In [0]:
import tensorflow as tf
import numpy as np

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# parse label data

def one_hot(oringinal_label):
  datalength = len(oringinal_label)
  label = np.zeros([datalength,10])
  for i in range(len(oringinal_label)):
    index = oringinal_label[i] 
    label[i,index]=1    
  return label


train_labels = one_hot(train_labels)
eval_labels = one_hot(eval_labels)

In [4]:
eval_labels[100,:]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [0]:
# define accuracy
def accuracy(labels, pred):
  # both labels and pred are in the shape of one-hot encoding
  accu=0
  for i in range(len(labels)):
    lab_val = np.argmax(labels[i,:])
    pred_val = np.argmax(pred[i,:])
    if lab_val==pred_val:
      accu+=1
  return accu/len(labels)


In [0]:
# hidden layer
def add_layer(inputs, in_size, out_size, rate, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size],stddev=0.01))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, rate)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    outputs = tf.nn.dropout(outputs, rate)
    # rate=1-keep_prop
    return outputs

In [0]:
# convolutional and pooling layer
def conv_layer(inputs, inputs_shape, nfilters):
  new_inputs = tf.reshape(inputs, inputs_shape)
  layer_output = tf.layers.conv2d(new_inputs, filters=nfilters, kernel_size=[5,5], padding='same')
  return layer_output


def maxpool(inputs):
  layer_output = tf.nn.max_pool(inputs, [1,2,2,1], [1,2,2,1], 'VALID')
  return layer_output

In [0]:
InputNeurons = 784
OutputNeurons = 10
drop_rate = 0.5  # dropout parameter, keep how many percentage
learning_rate = 0.01
printstep = 50
traing_epochs = 5001

# define placeholder for inputs to network
rate = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, InputNeurons])
ys = tf.placeholder(tf.float32, [None, OutputNeurons])

In [8]:
# neural network structure
conv1 = conv_layer(xs, [-1,28,28,1], 9)
mp1 = maxpool(conv1)
conv2 = conv_layer(mp1, [-1,14,14,1], 9)
mp2 = maxpool(conv2)
mp2 = tf.reshape(mp2, [-1,7*7*9*9])
l1 = add_layer(mp2, 7*7*9*9, 100, rate, activation_function=tf.nn.relu)
logits = add_layer(l1, 100, OutputNeurons, rate, activation_function=None)
prediction = tf.nn.softmax(logits)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# the error between prediction and real data
#loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), axis=1))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ys, logits=logits))
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

# start of the neural network
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(traing_epochs):
    # training
    idx = np.random.choice(np.arange(len(train_data)), 64, replace=False)
    x_train_batch = train_data[idx]
    y_train_batch = train_labels[idx]
    sess.run(train_step, feed_dict={xs: x_train_batch, ys: y_train_batch, rate: drop_rate})
    if i % printstep == 0:
        # to see the step improvement
        train_loss = sess.run(loss, feed_dict={xs: x_train_batch, ys: y_train_batch, rate: 1})
        validation_loss = sess.run(loss, feed_dict={xs: eval_data, ys: eval_labels, rate:1})
        print("epoch",i,"validation loss=", validation_loss,"Train Loss=",train_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

epoch 0 validation loss= 2.302743 Train Loss= 2.300772
epoch 50 validation loss= 2.298849 Train Loss= 2.297701
epoch 100 validation loss= 2.2948644 Train Loss= 2.2936587
epoch 150 validation loss= 2.2862446 Train Loss= 2.2803488
epoch 200 validation loss= 2.2715843 Train Loss= 2.2691715
epoch 250 validation loss= 2.2375126 Train Loss= 2.2359164
epoch 300 validation loss= 2.1545868 Train Loss= 2.1499424
epoch 350 validation loss= 2.0093887 Train Loss= 1.9957831
epoch 400 validation loss= 1.8725879 Train Loss= 1.8647704
epoch 450 validation loss= 1.7029461 Train Loss= 1.7338867
epoch 500 validation loss= 1.6413615 Train Loss= 1.7331532
epoch 550 validation loss= 1.5715687 Train Loss= 1.603802
epoch 600 validation loss= 1.4853343 Train Loss= 1.5517092
epoch 650 validation loss= 1.4588418 Train Loss= 1

In [0]:
eval_prediction = sess.run(prediction, feed_dict={xs: eval_data, ys: eval_labels, rate: 1})

In [21]:
print('vaildation data accuracy: ', accuracy(eval_labels, eval_prediction))

vaildation data accuracy:  0.9554
